Create a jupyter notebook or python script named spark101 for this exercise.

In [1]:
import pyspark
import pandas as pd
import numpy as np

from pydataset import data
from vega_datasets import data

from pyspark.sql.functions import col, expr, concat, sum, avg, min, max, count, mean, round 
from pyspark.sql.functions import lit, regexp_extract, regexp_replace, when, asc, desc, month, year, quarter
pd.set_option('display.max_colwidth', 10000)

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language

In [6]:
# Create Dataframe
lang_df = pd.DataFrame([['yes', 'no', 'yes', 'no', 'no', 'yes'], 
                        ['yes', 'no', 'yes', 'yes', 'no', 'yes'],
                       ], 
                        index = ['csv', 'json'], 
                        columns = ['python', 'javascript', 'c#', 'java', 'css', 'html'])
lang_df

,python,javascript,c#,java,css,html
csv,yes,no,yes,no,no,yes
json,yes,no,yes,yes,no,yes


In [7]:
# Convert to spark
df = spark.createDataFrame(lang_df)
df.show()

+------+----------+---+----+---+----+
|python|javascript| c#|java|css|html|
+------+----------+---+----+---+----+
|   yes|        no|yes|  no| no| yes|
|   yes|        no|yes| yes| no| yes|
+------+----------+---+----+---+----+



- View the schema of the dataframe

In [9]:
df.printSchema()

root
 |-- python: string (nullable = true)
 |-- javascript: string (nullable = true)
 |-- c#: string (nullable = true)
 |-- java: string (nullable = true)
 |-- css: string (nullable = true)
 |-- html: string (nullable = true)



- Output the shape of the dataframe

In [11]:
print((df.count(), len(df.columns)))

- Show the first 5 records in the dataframe

In [11]:
df.show(5)

(2, 6)
+------+----------+---+----+---+----+
|python|javascript| c#|java|css|html|
+------+----------+---+----+---+----+
|   yes|        no|yes|  no| no| yes|
|   yes|        no|yes| yes| no| yes|
+------+----------+---+----+---+----+



Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:
        The 1999 audi a4 has a 4 cylinder engine.
- For each vehicle:
    - Transform the trans column so that it only contains either manual or auto.

Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to farenheight.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).`m